In [1]:
# onnx로 바꾸기

# 신경망 함수 불러오기 (torch)
# onnx로 변환하기 (torch.onnx)
# 잘 저장 되었는지 확인하기 (onnx, onnxrunetime)

In [2]:
# Module import

import torch
import torch.nn as nn
import torch.onnx
import onnx
import onnxruntime

In [3]:
# 신경망 함수 NN.py import

from NN import PositionEncoding
from NN import Encoder

In [4]:
# 신경망 load

encoder = torch.load("encoder.pt", weights_only=False, map_location="cpu")
print(encoder)

Encoder(
  (embedding): Embedding(30187, 200, padding_idx=0)
  (pos_encoder): PositionEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
    )
    (linear1): Linear(in_features=200, out_features=1024, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=1024, out_features=200, bias=True)
    (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
       

In [5]:
# onnx로 변환하기
x = torch.randint(0, 30187, (1,30)).type(torch.long) # 더미 입력값, 문장하나 & 단어 25개
dynamic_axes = {'x' : {0 : 'b', 1 : 'f'}, 'y' : {0 : 'b'}}

encoder.eval() #혹시 모르니 명시적으로 dropout은 꺼주자
torch.onnx.export(
    encoder, # 바꿀 torch 신경망
    x, # 더미 입력값
    "encoder.onnx", # onnx 저장 파일 경로 
    export_params=True, # weight, bias 값들 저장할지
    opset_version=20,
    do_constant_folding=True,
    input_names=['x'],
    output_names=['y'],
    dynamic_axes=dynamic_axes, # 가변길이 차원
    dynamo=True
)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_784\2608939146.py:6: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(


[torch.onnx] Obtain model graph for `Encoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `Encoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 20 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 20},
            producer_name='pytorch',
            producer_version='2.7.0+cu126',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"x"<INT64,[1,s0]>
            ),
            outputs=(
                %"y"<FLOAT,[1,7]>
            ),
            initializers=(
                %"embedding.weight"<FLOAT,[30187,200]>{TorchTensor(...)},
                %"encoder.layers.0.self_attn.in_proj_weight"<FLOAT,[600,200]>{TorchTensor(...)},
                %"encoder.layers.0.self_attn.in_proj_bias"<FLOAT,[600]>{TorchTensor(...)},
                %"encoder.layers.0.self_attn.out_proj.weight"<FLOAT,[200,200]>{TorchTensor(...)},
                %"encoder.layers.0.self_attn.out_proj.bias"<FLOAT,[200]>{TorchTensor(...)},
                %"encoder.layers.0.linear1.weight"<FLOAT,[1024,200]>{TorchTensor(...)},
  

In [6]:
onnx_encoder = onnx.load('encoder.onnx')

onnx.checker.check_model(onnx_encoder) # 문제가 없으면 아무 메세지 출력하지 않음

In [14]:
import numpy as np

onnx_F = onnxruntime.InferenceSession("encoder.onnx", providers=["CPUExecutionProvider"])

np_x = np.random.randint(0, 30187, (1,22)).astype(np.int64) #문장 2개에 각 문장당 단어 10개
tensor_x = torch.tensor(np_x, dtype = torch.long)

tensor_y = encoder(tensor_x)
print(tensor_y.shape)

inputs = {onnx_F.get_inputs()[0].name : np_x}
outs = onnx_F.run(None, inputs)
np_y = outs[0]
print(np_y.shape)


np.testing.assert_allclose(tensor_y.detach().numpy(), np_y, rtol=1e-03, atol=1e-05) #오류 메세지가 없으면 오차범위 내에서만 차이가 있는 것

torch.Size([1, 7])
(1, 7)
